<a href="https://colab.research.google.com/github/sowmyamanojna/CS6910-Deep-Learning-Assignment-2/blob/main/Part-B/Part_B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Part-C: YOLOv3 for Forest Fire Detection

## Importing necessary packages

In [1]:
!pip install -q wandb
import wandb
import pathlib
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, Model
from tensorflow.keras.layers.experimental.preprocessing import Rescaling
from PIL import Image

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Loading the Training Data, Split Validation data

In [3]:
################################################################
# Preparing training (without augmentation) and validation set 
################################################################
# Preparing training and validation sets without augmentation
# Loading data from directory
# data_dir = pathlib.Path('/content/drive/MyDrive/inaturalist_12K/train') # Set path to the right directory
data_dir = '/content/drive/MyDrive/inaturalist_12K/train' # Set path to the right directory
train_data = tf.keras.preprocessing.image_dataset_from_directory(
                      directory = data_dir,
                      labels = 'inferred',  
                      label_mode = 'categorical',
                      color_mode = 'rgb',
                      batch_size = 32,
                      image_size = (256, 256),
                      shuffle = True,
                      seed = 17,
                      validation_split = 0.2,
                      subset = 'training')

val_data = tf.keras.preprocessing.image_dataset_from_directory(
                      directory = data_dir,
                      labels = 'inferred',  
                      label_mode = 'categorical',
                      color_mode = 'rgb',
                      batch_size = 32,
                      image_size = (256, 256),
                      shuffle = True,
                      seed = 17,
                      validation_split = 0.2,
                      subset = 'validation')

# Retaining 25 percent of train and validation data and discarding the rest
len_train, len_val = len(train_data), len(val_data)
train_data = train_data.take(int(0.25*len_train))
val_data = val_data.take(int(0.25*len_val))

Found 10000 files belonging to 10 classes.
Using 8000 files for training.
Found 10000 files belonging to 10 classes.
Using 2000 files for validation.


In [4]:
################################################################
# Preparing training set with augmentation 
################################################################
train_data_augmenter = ImageDataGenerator(
                            rescale = None,
                            rotation_range = 20,
                            width_shift_range = 0.2,
                            height_shift_range = 0.2,
                            brightness_range = [0.2, 1.5],
                            shear_range = 0.2,
                            zoom_range = 0.2,
                            horizontal_flip=True,
                            data_format = 'channels_last',
                            validation_split = 0.2)        #Specifying parameters for augmentation of training data

val_data_augmenter = ImageDataGenerator(validation_split = 0.2) #No augmentation of validation data

train_aug_gen = train_data_augmenter.flow_from_directory(data_dir, shuffle = True, \
                                                         seed = 17, subset = 'training')
val_aug_gen = val_data_augmenter.flow_from_directory(data_dir, shuffle = True, \
                                                     seed = 17, subset = 'validation')

# train_aug_data = tf.data.Dataset.from_generator(
#                     lambda: train_aug_gen,
#                     output_types = (tf.float32, tf.float32),
#                     output_shapes = ([None, 256, 256, 3], [None, 10]))

# val_aug_data = tf.data.Dataset.from_generator(
#                   lambda: val_aug_gen,
#                   output_types = (tf.float32, tf.float32),
#                   output_shapes = ([None, 256, 256, 3], [None, 10]))

# Retaining 25 percent of train and validation data and discarding the rest
# train_aug_data = train_aug_data.take(int(0.25*len_train))
# val_aug_data = val_aug_data.take(int(0.25*len_val))

Found 8000 images belonging to 10 classes.
Found 2000 images belonging to 10 classes.


Using tutorials:
- https://www.tensorflow.org/tutorials/text/image_captioning
- https://www.tensorflow.org/tutorials/images/transfer_learning


## Building the `CNN` class  that ensures modularity of code 

In [5]:
class CNN(Model):
    def __init__(self, base_model_name, tune=False, offset=20):
        super(CNN, self).__init__()
        self.IMG_SHAPE = (256, 256, 3)
        self.base_model_name = base_model_name
        self.init_base_model()
        
        if tune:
            self.base_model.trainable = True
            fine_tune_at = len(self.base_model.layers)-offset
            # Freeze all the layers before the `fine_tune_at` layer
            for layer in self.base_model.layers[:fine_tune_at]:
                layer.trainable =  False
        else:
            self.base_model.trainable = False

        self.additional_avg_pool1 = layers.GlobalAveragePooling2D()
        self.additional_dense1 = layers.Dense(10)

    def init_base_model(self):
        if self.base_model_name == "InceptionV3":
            self.base_model = tf.keras.applications.InceptionV3(input_shape=self.IMG_SHAPE,
                                               include_top=False,
                                               weights='imagenet')
        elif self.base_model_name == "InceptionResNetV2":
            self.base_model = tf.keras.applications.InceptionResNetV2(input_shape=self.IMG_SHAPE,
                                               include_top=False,
                                               weights='imagenet')
        elif self.base_model_name == "ResNet50":
            self.base_model = tf.keras.applications.ResNet50(input_shape=self.IMG_SHAPE,
                                               include_top=False,
                                               weights='imagenet')
        elif self.base_model_name == "Xception":
            self.base_model = tf.keras.applications.Xception(input_shape=self.IMG_SHAPE,
                                               include_top=False,
                                               weights='imagenet')
            
    def call(self, inputs):
        x = self.base_model(inputs)
        x = self.additional_avg_pool1(x)
        return self.additional_dense1(x)

## Training without wandb

In [6]:
base_model_name = "ResNet50"
tune = False
offset = 20

model = CNN(base_model_name, tune, offset)

base_learning_rate = 0.0001
model.compile(optimizer=tf.keras.optimizers.Adam(lr=base_learning_rate),
                loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
                metrics=['accuracy'])

# history = model.fit(train_data, epochs=10, validation_data=val_data)
history = model.fit(train_aug_gen, steps_per_epoch=62, \
                    epochs=10, validation_data=val_aug_gen, \
                    validation_steps=62, workers=8)

Epoch 1/10
62/62 [==============================] - 78s 1s/step - loss: 2.9546 - accuracy: 0.1057 - val_loss: 2.5126 - val_accuracy: 0.1643
Epoch 2/10
62/62 [==============================] - 75s 1s/step - loss: 2.3557 - accuracy: 0.1684 - val_loss: 2.1185 - val_accuracy: 0.2611
Epoch 3/10
62/62 [==============================] - 74s 1s/step - loss: 2.0297 - accuracy: 0.2846 - val_loss: 1.8854 - val_accuracy: 0.3377
Epoch 4/10
62/62 [==============================] - 74s 1s/step - loss: 1.8514 - accuracy: 0.3548 - val_loss: 1.7115 - val_accuracy: 0.4052
Epoch 5/10
62/62 [==============================] - 74s 1s/step - loss: 1.7590 - accuracy: 0.3971 - val_loss: 1.6041 - val_accuracy: 0.4526
Epoch 6/10
62/62 [==============================] - 74s 1s/step - loss: 1.6568 - accuracy: 0.4395 - val_loss: 1.4925 - val_accuracy: 0.4975
Epoch 7/10
62/62 [==============================] - 74s 1s/step - loss: 1.5941 - accuracy: 0.4655 - val_loss: 1.4237 - val_accuracy: 0.5292
Epoch 8/10
62/62 [==

## Train with wandb

In [7]:
###############################################
# Listing the hyperparameters in wandb config 
###############################################
sweep_config = {'name': 'random-test-sweep', 'method': 'grid'}
sweep_config['metric'] = {'name': 'val_acc', 'goal': 'maximize'}
parameters_dict = {
                   'base_model_name': {'values': ["InceptionV3", "InceptionResNetV2", "ResNet50", "Xception"]},
                   'tune': {'values': [False, True]},
                   'data_aug': {'values': [False, True]}
                  }
sweep_config['parameters'] = parameters_dict

In [11]:
def pretrain_CNN_sweep(config=sweep_config):
    with wandb.init(config=config):
        config = wandb.init().config
        wandb.run.name = 'BM_{}_tune_{}_aug_{}'.format(config.base_model_name, \
                                                       config.tune, config.data_aug)
        
        model = CNN(config.base_model_name, config.tune)
        base_learning_rate = 0.0001
        model.compile(optimizer=tf.keras.optimizers.Adam(lr=base_learning_rate),
                loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
                metrics=['accuracy'])
        
        if config.data_aug == False:
            history = model.fit(train_data, epochs=10, validation_data=val_data, \
                            callbacks = [wandb.keras.WandbCallback()])
        else:
            history = model.fit(train_aug_gen, steps_per_epoch=62, epochs=10, \
                                validation_data=val_aug_gen, validation_steps=62,\
                                workers=10, callbacks = [wandb.keras.WandbCallback()])

In [12]:
#################################
# Setting up wandb sweeps
#################################
sweep_id = wandb.sweep(sweep_config, project = 'DL-Assignment2-PartB')
wandb.agent(sweep_id, function=pretrain_CNN_sweep)

Create sweep with ID: wdu7jyge
Sweep URL: https://wandb.ai/cs6910-team/DL-Assignment2-PartB/sweeps/wdu7jyge


wandb: Agent Starting Run: fldgqaem with config:
wandb: 	base_model_name: InceptionV3
wandb: 	data_aug: False
wandb: 	tune: False


Epoch 1/10
62/62 [==============================] - 36s 483ms/step - loss: 21.9916 - accuracy: 0.0924 - val_loss: 11.0436 - val_accuracy: 0.1271


wandb: ERROR Can't save model, h5py returned error: Saving the model to HDF5 format requires the model to be a Functional model or a Sequential model. It does not work for subclassed models, because such models are defined via the body of a Python method, which isn't safely serializable. Consider saving to the Tensorflow SavedModel format (by setting save_format="tf") or using `save_weights`.


Epoch 2/10
62/62 [==============================] - 30s 459ms/step - loss: 10.5274 - accuracy: 0.1083 - val_loss: 8.6774 - val_accuracy: 0.1292
Epoch 3/10
62/62 [==============================] - 30s 459ms/step - loss: 8.8910 - accuracy: 0.1057 - val_loss: 8.2379 - val_accuracy: 0.1375
Epoch 4/10
62/62 [==============================] - 30s 455ms/step - loss: 7.8416 - accuracy: 0.1204 - val_loss: 7.3140 - val_accuracy: 0.1333
Epoch 5/10
62/62 [==============================] - 31s 463ms/step - loss: 7.2795 - accuracy: 0.1164 - val_loss: 7.0561 - val_accuracy: 0.1187
Epoch 6/10
62/62 [==============================] - 30s 459ms/step - loss: 6.5678 - accuracy: 0.1263 - val_loss: 6.7489 - val_accuracy: 0.1312
Epoch 7/10
62/62 [==============================] - 31s 463ms/step - loss: 6.2510 - accuracy: 0.1343 - val_loss: 6.3339 - val_accuracy: 0.1500
Epoch 8/10
62/62 [==============================] - 30s 458ms/step - loss: 5.8442 - accuracy: 0.1472 - val_loss: 5.9909 - val_accuracy: 0.141

...

wandb: Agent Starting Run: ba3h8ard with config:
wandb: 	base_model_name: Xception
wandb: 	data_aug: True
wandb: 	tune: True


Epoch 1/10
62/62 [==============================] - 92s 1s/step - loss: 2.2913 - accuracy: 0.1185 - val_loss: 2.2749 - val_accuracy: 0.1421


wandb: ERROR Can't save model, h5py returned error: Saving the model to HDF5 format requires the model to be a Functional model or a Sequential model. It does not work for subclassed models, because such models are defined via the body of a Python method, which isn't safely serializable. Consider saving to the Tensorflow SavedModel format (by setting save_format="tf") or using `save_weights`.


Epoch 2/10
62/62 [==============================] - 88s 1s/step - loss: 2.2507 - accuracy: 0.1648 - val_loss: 2.2529 - val_accuracy: 0.1789
Epoch 3/10
62/62 [==============================] - 88s 1s/step - loss: 2.2255 - accuracy: 0.1760 - val_loss: 2.1917 - val_accuracy: 0.2031
Epoch 4/10
62/62 [==============================] - 87s 1s/step - loss: 2.1741 - accuracy: 0.2256 - val_loss: 2.2114 - val_accuracy: 0.1704
Epoch 5/10
62/62 [==============================] - 87s 1s/step - loss: 2.1823 - accuracy: 0.1905 - val_loss: 2.1798 - val_accuracy: 0.2167
Epoch 6/10
62/62 [==============================] - 87s 1s/step - loss: 2.1656 - accuracy: 0.2255 - val_loss: 2.1629 - val_accuracy: 0.2203
Epoch 7/10
62/62 [==============================] - 87s 1s/step - loss: 2.1534 - accuracy: 0.2219 - val_loss: 2.1759 - val_accuracy: 0.1991
Epoch 8/10
62/62 [==============================] - 87s 1s/step - loss: 2.1427 - accuracy: 0.2112 - val_loss: 2.2195 - val_accuracy: 0.2122
Epoch 9/10
62/62 [==

wandb: Sweep Agent: Waiting for job.
wandb: Sweep Agent: Exiting.
